In [8]:
import rospy
import actionlib
import PyKDL
import tf2_ros

from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
from geometry_msgs.msg import Twist, Pose, Pose2D, PoseStamped
from nav_msgs.msg import Odometry, Path
from gazebo_msgs.msg import ModelState, ContactsState

from threading import Event

In [ ]:
class RosBridge:
    
    def __init__(self, real_robot=False):
        
        # Event is clear while initialization or set_state is going on
        self.reset = Event()
        self.reset.clear()
        self.get_state_event()
        self.get_state_event.set()
        
        self.real_robot = real_robot
        
        self.move_base_client = actionlib.SimpleActionClient('/move_base', MoveBaseAction)
        self.mir_exec_path = rospy.Publisher('mir_exec_path', Path, queue_size=10)
        
        if self.real_robot:
            rospy.Subscriber("odom", Odometry, self.callbackOdometry, queue_size=1)
        else:
            rospy.Subscriber("odom_comb", Odometry, self.callbackOdometry, queue_size=1)
            
        rospy.Subscriber("mir_collision", ContactsState, self.collision_callback)
        
        self.mir_pose = [0.0] * 3
        self.mir_twist = [0.0] * 2
        self.collision = False
        
        self.obstacles = [] # [3,n]
        self.targets = [] # [3,n]
        
        self.path_frame = 'map'
        
        if self.real_robot:
            self.path_frame = 'world'

            # Apply transform to center the robot, with real_robot we use World frame,
            # World is the Map frame translated in XY to center robot
            tfBuffer = tf2_ros.Buffer()
            listener = tf2_ros.TransformListener(tfBuffer)

            trans = tfBuffer.lookup_transform("world", "map", rospy.Time(), rospy.Duration(1.0))
            v = PyKDL.Vector(trans.transform.translation.x,trans.transform.translation.y,trans.transform.translation.z)
            r = PyKDL.Rotation.Quaternion(trans.transform.rotation.x,trans.transform.rotation.y,trans.transform.rotation.z,trans.transform.rotation.w)
            self.world_to_map = PyKDL.Frame(r,v)
        
        rospy.Subscriber("robot_pose", Pose, self.callbackState, queue_size=1)
        
        # Initialize Path
        self.mir_path = Path()
        self.mir_path.header.stamp = rospy.Time.now()
        self.mir_path.header.frame_id = self.path_frame

        # Flag indicating if it is safe to move backwards
        self.safe_to_move_back = True
        # Flag indicating if it is safe to move forward
        self.safe_to_move_front = True
        self.rate = rospy.Rate(10) #30Hz
        self.reset.set()